In [1]:
import pandas as pd
import numpy as np
from nptdms import TdmsFile
import tensorflow as tf
import os

2025-06-04 12:34:48.642273: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-04 12:34:48.644855: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-04 12:34:48.652767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749033288.665814   41196 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749033288.669785   41196 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749033288.679852   41196 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
def create_dataset(
        seq_length: int, #5000
        dataset_folder: str,
        tdms_file: str
):
        Data = TdmsFile.read(tdms_file).as_dataframe()
        cols = Data.columns
        X_cols = [x for x in cols if x.endswith('ai0\'')]
        Y_cols = [x for x in cols if x.endswith('ai1\'')]

        X = Data[X_cols].to_numpy().flatten()
        Y = Data[Y_cols].to_numpy().flatten()

        DATASET_SIZE = int(X.shape[0] / seq_length)

        mean_x = np.mean(X)
        std_x = np.std(X)
        X = (X-mean_x)/std_x

        # np.save(
        #         os.path.join(dataset_folder, f"{os.path.split(tdms_file)[-1].split("_2025")[0]}.npy"),
        #         np.array([mean_x, std_x])
        # )

        X = np.expand_dims(X, axis = 1)
        Y = np.expand_dims(Y, axis = 1)

        dataset = tf.data.Dataset.from_tensor_slices(((X, tf.zeros_like(Y)), Y))
        dataset = dataset.batch(seq_length, drop_remainder=True)

        train_size = int(0.7 * DATASET_SIZE)
        val_size = int(0.15 * DATASET_SIZE)
        test_size = int(0.15 * DATASET_SIZE)

        full_dataset = dataset #tf.data.TFRecordDataset(FLAGS.input_file)
        full_dataset = full_dataset.shuffle(buffer_size=DATASET_SIZE)
        train_dataset = full_dataset.take(train_size)
        test_dataset = full_dataset.skip(train_size)
        val_dataset = test_dataset.skip(val_size)
        test_dataset = test_dataset.take(test_size)

        train_dataset.save(os.path.join(dataset_folder, "train_" + str(os.path.split(tdms_file)[-1].split(".tdms")[0])))
        test_dataset.save(os.path.join(dataset_folder, "test_" + str(os.path.split(tdms_file)[-1].split(".tdms")[0])))
        val_dataset.save(os.path.join(dataset_folder, "val_" + str(os.path.split(tdms_file)[-1].split(".tdms")[0])))
        # print("Succesfully saved:", f"{os.path.split(tdms_file)[-1].split("_2025")[0]}")

In [21]:
datasets = ['Data_processes_2025_06_03.tdms',
            'Data_diffent_process_params_2025_06_03.tdms',
            'LongOUData.tdms',
            'LongOU.tdms']
N_drive = '/mnt/N/SCI-NBI-quantop-data/data/gwd/Experimental Data/atoms/2025/2025-05-28 - Chistian Jun ML DAQ'

dataset_folder = os.path.join(N_drive, 'ML_datasets')


In [ ]:
df = pd.DataFrame()
for i in range(2):
    colname = datasets[i].removesuffix("_2025_06_03.tdms").removeprefix("Data_")
    fileread = TdmsFile.read(os.path.join(N_drive, datasets[i])).as_dataframe()
    df["var" + colname] = fileread.var()
    df["mean" + colname] = fileread.mean()
df.to_pickle('process_var_mean.pkl')


In [ ]:
df = pd.DataFrame()
for i in range(2, 4):
    colname = datasets[i].removesuffix(".tdms")
    fileread = TdmsFile.read(os.path.join(N_drive, datasets[i])).as_dataframe()
    df["var" + colname] = fileread.var()
    df["mean" + colname] = fileread.mean()
df.to_pickle('OU_var_mean.pkl')


In [4]:
for file in datasets:
    create_dataset(
        seq_length = 5000,
        dataset_folder=dataset_folder,
        tdms_file=os.path.join(N_drive, file)
    )

E0000 00:00:1749033303.049882   41196 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1749033303.052972   41196 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-06-04 12:35:15.529734: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 1814 of 29000
2025-06-04 12:35:25.533060: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 3539 of 29000
2025-06-04 12:35:35.535068: I tensorflow/core/kernels/dat